In [1]:
# from __future__ import print_function
# import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import log_loss
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
import keras
from keras import regularizers
from collections import Counter
# %matplotlib inline
np.random.seed(1671) # for reproducibility
# from help_function import LoadData

Using TensorFlow backend.
D:\Users\X1Carbon\Anaconda3\envs\TensorFlow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\X1Carbon\Anaconda3\envs\TensorFlow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\X1Carbon\Anaconda3\envs\TensorFlow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Users\X1Carbon\Anaco

In [8]:
data.columns

Index(['SaleID', 'bodyType', 'brand', 'brand_type', 'city', 'creatDate',
       'fuelType', 'gearbox', 'is_train', 'kilometer', 'model', 'name',
       'notRepairedDamage', 'power', 'price', 'regDate', 'regionCode', 'v_0',
       'v_1', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'v_2', 'v_3', 'v_4',
       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'regDate_year', 'creatDate_year',
       'during_month', 'during_year', 'brand_amount', 'brand_price_average',
       'brand_price_max', 'brand_price_median', 'brand_price_min',
       'brand_price_std', 'brand_price_sum', 'brand_type_amount',
       'brand_type_average', 'brand_type_max', 'brand_type_median',
       'brand_type_min', 'brand_type_std', 'brand_type_sum', 'city_amount',
       'city_average', 'city_max', 'city_median', 'city_min', 'city_std',
       'city_sum'],
      dtype='object')

In [34]:
data = pd.read_csv("nn_data.csv")


train_data = data[data['is_train']==1]
test_data = data[data['is_train'] ==0]
#test = test[test['mt'] <25]
train_data.head()

,SaleID,bodyType,brand,brand_type,city,creatDate,fuelType,gearbox,is_train,kilometer,...,brand_type_min,brand_type_std,brand_type_sum,city_amount,city_average,city_max,city_median,city_min,city_std,city_sum
0,0,1.0,6,601.0,10,2016-04-04,0.0,0.0,1,12.5,...,30.0,2707.072478,7868239.0,3521.0,6176.86,73900.0,3500.0,20.0,7397.792986,21754915.0
1,1,2.0,1,102.0,43,2016-03-09,0.0,0.0,1,15.0,...,100.0,7802.604925,46879429.0,1472.0,5449.87,42700.0,3200.0,20.0,6140.251744,8027664.0
2,2,1.0,15,1501.0,28,2016-04-02,0.0,0.0,1,12.5,...,100.0,4940.333864,7390310.0,2199.0,5546.22,63999.0,3200.0,30.0,6613.670703,12201689.0
3,3,0.0,10,1000.0,4,2016-03-12,0.0,1.0,1,15.0,...,35.0,6962.900499,37761446.0,3905.0,6226.05,89950.0,3700.0,20.0,7120.912973,24318967.0
4,4,1.0,5,501.0,69,2016-03-13,0.0,0.0,1,5.0,...,50.0,2045.355745,4016555.0,535.0,5757.79,48500.0,3450.0,50.0,6733.317718,3086176.0


In [35]:
# data = pd.read_csv("nn_data1010.csv")

# del data['forecastVolum'],data['pred_label']
# train_data = data[data['mt']<25]
# test = data[data['mt']>=25]
# train_data.head()


## load data
# train_data = pd.read_csv('data/train.csv')
# test_data = pd.read_csv('data/test.csv')
epochs = 100
batch_size = 512
classes = 1

## category feature one_hot
test_data['price'] = -1


num_feats = ['power', 'kilometer', 'during_month',  'during_year','brand_amount',
       'brand_price_average', 'brand_price_max', 'brand_price_median',
       'brand_price_min', 'brand_price_std', 'brand_price_sum','city',
       'brand_type_amount', 'brand_type_average', 'brand_type_max',
       'brand_type_median', 'brand_type_min', 'brand_type_std','regDate_year', 
       'brand_type_sum',
       'city_amount', 
       'city_average','city_median', 'city_min', 'city_std',
       'city_sum',
       'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_9','v_5',
       'v_8',  'v_10', 'v_11', 'v_12', 'v_13','v_14' ] # 'v_6', 'v_7','v_9', 'v_5','v_6','v_11', 'city_max', 

cate_feats = ['model', 'brand', 'bodyType', 'fuelType','name','gearbox',
             'notRepairedDamage', 'regionCode','brand_type']

feats = cate_feats+num_feats

for item in cate_feats:#
    data[item] = LabelEncoder().fit_transform(data[item])
    
train_data = data[data['is_train']==1]
test_data = data[data['is_train'] ==0]    
print(data.shape)
print(train_data.shape)
print(test_data.shape)

'''
for item in cate_feats:
    print(item)
    data[item] = LabelEncoder().fit_transform(data[item])
    item_dummies = pd.get_dummies(data[item])
    item_dummies.columns = [item + str(i + 1) for i in range(item_dummies.shape[1])]
    data = pd.concat([data, item_dummies], axis=1)
data.drop(cate_feature,axis=1,inplace=True)
'''

D:\Users\X1Carbon\Anaconda3\envs\TensorFlow\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(199037, 57)
(149037, 57)
(50000, 57)


'\nfor item in cate_feats:\n    print(item)\n    data[item] = LabelEncoder().fit_transform(data[item])\n    item_dummies = pd.get_dummies(data[item])\n    item_dummies.columns = [item + str(i + 1) for i in range(item_dummies.shape[1])]\n    data = pd.concat([data, item_dummies], axis=1)\ndata.drop(cate_feature,axis=1,inplace=True)\n'

In [32]:
train_data

,SaleID,bodyType,brand,brand_type,city,creatDate,fuelType,gearbox,is_train,kilometer,...,brand_type_min,brand_type_std,brand_type_sum,city_amount,city_average,city_max,city_median,city_min,city_std,city_sum
0,0,1,6,44,10,2016-04-04,0,0.0,1,12.5,...,30.0,2707.072478,7868239.0,3521.0,6176.86,73900.0,3500.0,20.0,7397.792986,21754915.0
1,1,2,1,10,43,2016-03-09,0,0.0,1,15.0,...,100.0,7802.604925,46879429.0,1472.0,5449.87,42700.0,3200.0,20.0,6140.251744,8027664.0
2,2,1,15,115,28,2016-04-02,0,0.0,1,12.5,...,100.0,4940.333864,7390310.0,2199.0,5546.22,63999.0,3200.0,30.0,6613.670703,12201689.0
3,3,0,10,75,4,2016-03-12,0,1.0,1,15.0,...,35.0,6962.900499,37761446.0,3905.0,6226.05,89950.0,3700.0,20.0,7120.912973,24318967.0
4,4,1,5,36,69,2016-03-13,0,0.0,1,5.0,...,50.0,2045.355745,4016555.0,535.0,5757.79,48500.0,3450.0,50.0,6733.317718,3086176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149032,149995,4,10,79,45,2016-03-27,0,1.0,1,15.0,...,399.0,9308.178506,15432706.0,1373.0,5906.85,89000.0,3500.0,60.0,7054.100166,8116007.0
149033,149996,0,11,83,28,2016-03-12,0,0.0,1,10.0,...,80.0,5176.965741,4882439.0,2199.0,5546.22,63999.0,3200.0,30.0,6613.670703,12201689.0
149034,149997,1,11,84,33,2016-03-28,1,0.0,1,6.0,...,45.0,2984.005274,4433411.0,2108.0,5810.35,72900.0,3199.5,15.0,7257.073663,12254030.0
149035,149998,3,10,78,18,2016-04-01,1,0.0,1,15.0,...,300.0,8332.005838,11921249.0,3134.0,6094.26,99999.0,3450.0,30.0,7673.683635,19105495.0


In [36]:
target = ['price']
'''
sparse_features = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'D1', 'D2', 'E1',
       'E11', 'E12',  'E15', 'E18',  # 'E10','E13', 'E14','E16', 'E17', 'E19', 'E2','E21','E22','E3','E5','E7','E9',
       'E20',   'E23', 'E24', 'E25', 'E26', 'E27', 'E28','E16', 'E17', 'E19', 'E2', 'E21', 'E22', 'E3', 'E5', 'E7', 'E9',
       'per_day_hour_click_rate', 'per_hour_click_rate', 'AB_click_rate', 'BD1_click_rate', 'BD2_click_rate',
       'E29',  'E4',  'E6',  'E8',   'day',
        'shijianduan', 'time', 'weekend',]#
       #'per_day_hour_click_rate', 'per_day_click_rate', 'per_hour_click_rate']

notfeature = [ 'ID','date_index','date','week','before_day_hour_click_rate','after_day_hour_click_rate','week_day_hour_click_rate']
'''
not_features = [i for i in train_data.columns if i not in feats]




print(not_features)

['SaleID', 'creatDate', 'is_train', 'price', 'regDate', 'v_6', 'v_7', 'creatDate_year', 'city_max']


In [41]:
train_x = train_data[feats]
train_y = train_data['price'].values
test = test[feats]

scaler = StandardScaler()
train_X = pd.DataFrame()
test_X = pd.DataFrame()
train_X[num_feats] = scaler.fit_transform(train_x[num_feats])
test_X[num_feats] = scaler.transform(test[num_feats])
'''
mms = MinMaxScaler(feature_range=(0, 1))
for feat in num_feats:
    print("Process: ", feat)
    data[feat] = mms.fit_transform(np.array(data[feat]).reshape(-1, 1))
print("Process dense finish ")
'''
# 2.count #unique features for each sparse field,and record dense feature field name



KeyError: "None of [Index(['power', 'kilometer', 'during_month', 'during_year', 'brand_amount',\n       'brand_price_average', 'brand_price_max', 'brand_price_median',\n       'brand_price_min', 'brand_price_std', 'brand_price_sum', 'city',\n       'brand_type_amount', 'brand_type_average', 'brand_type_max',\n       'brand_type_median', 'brand_type_min', 'brand_type_std', 'regDate_year',\n       'brand_type_sum', 'city_amount', 'city_average', 'city_median',\n       'city_min', 'city_std', 'city_sum', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4',\n       'v_9', 'v_5', 'v_8', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14'],\n      dtype='object')] are in the [columns]"

In [ ]:
folds = KFold(n_splits=2, shuffle=True, random_state=1996)
NN_predictions = np.zeros((test_X.shape[0], 1))
oof_preds = np.zeros((train_X.shape[0], 1))

patience = 100   ## How many steps to stop
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=100,
                                        mode='auto', baseline=None)

for fold_, (trn_, val_) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    x_train, y_train = train_X[trn_], train_y[trn_]
    x_valid, y_valid = train_X[val_], train_y[val_]


    model = MLP(dropout_rate=0.5, activation='relu')
    model.compile( Adam(lr=0.1, decay=0),loss='mse')#, metrics=['mse'],optimizer='adam',
    history = model.fit(x_train, y_train,
                        validation_data=[x_valid, y_valid],
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[call_ES, ],
                        shuffle=True,
                        verbose=1)

    # plot_loss_acc(history, fold_ + 1)

    print('Loading Best Model')
    # # Get predicted probabilities for each class
    oof_preds[val_] = model.predict(x_valid, batch_size=batch_size)
    NN_predictions += model.predict(test_X, batch_size=batch_size) / folds.n_splits

print(NN_predictions)

In [78]:
# data = pd.read_csv("nn_data1010.csv")

# del data['forecastVolum'],data['pred_label']
# train_data = data[data['mt']<25]
# test = data[data['mt']>=25]
# train_data.head()


## load data
# train_data = pd.read_csv('data/train.csv')
# test_data = pd.read_csv('data/test.csv')
epochs = 100
batch_size = 512
classes = 1

## category feature one_hot
test_data['price'] = -1


num_feats = ['power', 'kilometer', 'during_month',  'during_year','brand_amount',
       'brand_price_average', 'brand_price_max', 'brand_price_median',
       'brand_price_min', 'brand_price_std', 'brand_price_sum','city',
       'brand_type_amount', 'brand_type_average', 'brand_type_max',
       'brand_type_median', 'brand_type_min', 'brand_type_std','regDate_year', 
       'brand_type_sum',
       'city_amount',
       'city_average','city_median', 'city_min', 'city_std',
       'city_sum',
       'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_9','v_5',
       'v_8',  'v_10', 'v_11', 'v_12', 'v_13','v_14' ] # 'v_6', 'v_7','v_9', 'v_5','v_6','v_11', 'city_max', 

cate_feats = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox',
             'notRepairedDamage', 'regionCode','brand_type']

feats = cate_feats+num_feats

for item in cate_feats:
    data[item] = LabelEncoder().fit_transform(data[item])
    item_dummies = pd.get_dummies(data[item])
    item_dummies.columns = [item + str(i + 1) for i in range(item_dummies.shape[1])]
    data = pd.concat([data, item_dummies], axis=1)
data.drop(cate_feature,axis=1,inplace=True)

train = data[data['price'] != -1]
test = data[data['price'] == -1]

##Clean up the memory
# del data, train_data, test_data
# gc.collect()

## get train feature
# del_feature = [ 
#  'id',
#  'model_adcode',]
# features = [i for i in train.columns if i not in del_feature]


train_x = train[features]
train_y = train['label'].values
test = test[features]

## Fill missing value
# for i in train_x.columns:
#     # print(i, train_x[i].isnull().sum(), test[i].isnull().sum())
#     if train_x[i].isnull().sum() != 0:
#         train_x[i].fillna(0, inplace=True)
#         test[i].fillna(0, inplace=True)

## normalized
scaler = StandardScaler()
train_X = scaler.fit_transform(train_x)
test_X = scaler.transform(test)
# train_X  = train_x.copy()
# test_X   = test.copy()


# K.clear_session()



folds = KFold(n_splits=2, shuffle=True, random_state=1996)
NN_predictions = np.zeros((test_X.shape[0], 1))
oof_preds = np.zeros((train_X.shape[0], 1))

patience = 100   ## How many steps to stop
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=100,
                                        mode='auto', baseline=None)

for fold_, (trn_, val_) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    x_train, y_train = train_X[trn_], train_y[trn_]
    x_valid, y_valid = train_X[val_], train_y[val_]


    model = MLP(dropout_rate=0.5, activation='relu')
    model.compile( Adam(lr=0.1, decay=0),loss='mse')#, metrics=['mse'],optimizer='adam',
    history = model.fit(x_train, y_train,
                        validation_data=[x_valid, y_valid],
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[call_ES, ],
                        shuffle=True,
                        verbose=1)

    # plot_loss_acc(history, fold_ + 1)

    print('Loading Best Model')
    # # Get predicted probabilities for each class
    oof_preds[val_] = model.predict(x_valid, batch_size=batch_size)
    NN_predictions += model.predict(test_X, batch_size=batch_size) / folds.n_splits

print(NN_predictions)

D:\Users\X1Carbon\Anaconda3\envs\TensorFlow\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



fold 1
Train on 13200 samples, validate on 13200 samples
Epoch 1/100
13200/13200 [==============================] - 36s 3ms/step - loss: 668381.7466 - val_loss: 12063936.0339
Epoch 2/100
13200/13200 [==============================] - 4s 319us/step - loss: 368842.0299 - val_loss: 10874914.8445
Epoch 3/100
13200/13200 [==============================] - 5s 348us/step - loss: 345013.1106 - val_loss: 3612330.7148
Epoch 4/100
13200/13200 [==============================] - 13s 980us/step - loss: 327318.3086 - val_loss: 1874428.3529
Epoch 5/100
13200/13200 [==============================] - 4s 294us/step - loss: 316248.0072 - val_loss: 1571645.2358
Epoch 6/100
13200/13200 [==============================] - 3s 235us/step - loss: 315844.6537 - val_loss: 1492176.2415
Epoch 7/100
13200/13200 [==============================] - 3s 226us/step - loss: 315287.5698 - val_loss: 980435.9378
Epoch 8/100
13200/13200 [==============================] - 3s 216us/step - loss: 311399.2918 - val_loss: 892793.7068

Epoch 70/100
13200/13200 [==============================] - 3s 240us/step - loss: 235101.6600 - val_loss: 214638.9129
Epoch 71/100
13200/13200 [==============================] - 3s 237us/step - loss: 222319.6428 - val_loss: 240634.0804
Epoch 72/100
13200/13200 [==============================] - 3s 241us/step - loss: 234314.8885 - val_loss: 214832.8619
Epoch 73/100
13200/13200 [==============================] - 3s 227us/step - loss: 226345.9753 - val_loss: 261531.2490
Epoch 74/100
13200/13200 [==============================] - 4s 270us/step - loss: 274993.4141 - val_loss: 290747.0931
Epoch 75/100
13200/13200 [==============================] - 3s 241us/step - loss: 277937.8244 - val_loss: 257028.7673
Epoch 76/100
13200/13200 [==============================] - 3s 247us/step - loss: 271076.6811 - val_loss: 242807.6129
Epoch 77/100
13200/13200 [==============================] - 3s 248us/step - loss: 258422.9538 - val_loss: 225874.3178
Epoch 78/100
13200/13200 [==============================

KeyboardInterrupt: 

In [33]:
def MLP(dropout_rate=0.5, activation='relu'):
    start_neurons = 600
    model = Sequential()
    model.add(Dense(start_neurons, input_dim=train_X.shape[1], activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons , activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(start_neurons, activation=activation))#// 2
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(start_neurons// 2, activation=activation))#// 2
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))


    
    model.add(Dense(start_neurons// 4 , activation=activation))#// 4
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))


    model.add(Dense(start_neurons // 8, activation=activation))#// 8
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    


    model.add(Dense(classes, activation='linear'))
    return model

In [ ]:
loss: 230082.7578 - val_loss: 191146.1819
Loading Best Model

In [77]:
NN_predictions[:10]

array([[155.35545349],
       [192.12611389],
       [168.08506012],
       [260.80189514],
       [622.19866943],
       [173.06531525],
       [269.92922974],
       [165.67009735],
       [957.44317627],
       [177.01425171]])